# **California housing:** a Machine Learning implementation

In [ ]:
# Importera bibliotek
python -m pip install numpy pandas matplotlib seaborn scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Ladda in housing.csv
import sys, os 
sys.path.append(os.path.abspath(".."))
df = pd.read_csv("./data/housing.csv")